In [2]:
import pickle as pkl
import numpy as np 
import pandas as pd
import json

In [11]:
model = pkl.load(open('./model.pkl','rb'))
userdb = pkl.load(open('userdb.pkl','rb'))
articledb = pkl.load(open('articles.pkl','rb'))
vectorizer = pkl.load(open('./vectorizer.pkl','rb'))
classifier = pkl.load(open('./classifier.pkl','rb'))

In [17]:

df = pd.read_json('./data.json')
df.head(3)

,category,headline,authors,link,short_description,date,Text_cleaning
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,2018-05-26,mass shootings texas last week
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Andy McDonald,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.,2018-05-26,smith joins diplo nicky jam world cups officia...
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...,2018-05-26,hugh grant marries first time age


In [8]:
def vec(vectorizer,text):
    return vectorizer.transform([text])

def add_article(text):
    v = vec(vectorizer,text)
    preds = classifier.predict_proba(v)
    articledb.append(preds)
    

In [55]:
# def recommend():
def softmax(x):
    return np.exp(x) / np.sum(np.exp(x), axis=0)
    
user = np.random.multinomial(100,[1/41.]*41, size=1)
user = np.expand_dims(softmax(user[0]),axis=0)
model.predict(user)
# model.cluster_centers_[4]

from sklearn.neighbors import NearestNeighbors
nn =NearestNeighbors(n_neighbors=3,algorithm = 'ball_tree')
# knn_colabarative = KNeighborsClassifier(n_neighbors=3)

nn.fit(articledb)



NearestNeighbors(algorithm='ball_tree', n_neighbors=3)

In [56]:
user

array([[0.00449774, 0.00449774, 0.01222612, 0.03323405, 0.00449774,
        0.00165463, 0.00165463, 0.03323405, 0.00449774, 0.0006087 ,
        0.00449774, 0.00165463, 0.00165463, 0.01222612, 0.03323405,
        0.24556825, 0.09033951, 0.03323405, 0.09033951, 0.03323405,
        0.00165463, 0.01222612, 0.01222612, 0.00165463, 0.0006087 ,
        0.00165463, 0.00449774, 0.00165463, 0.00165463, 0.00449774,
        0.00165463, 0.09033951, 0.00165463, 0.00165463, 0.01222612,
        0.00165463, 0.01222612, 0.09033951, 0.00449774, 0.00449774,
        0.09033951]])

In [ ]:
pkl.dump(nn,open('./nearest_neighbour.pkl','wb'))

In [57]:
# rec = pd.DataFrame()


def recommend (user):
    links = []
    headlines=[]
    ids = []
    cluster = model.predict(user)[0]
    collab_user =np.expand_dims(model.cluster_centers_[cluster],axis=0)
    dist,ind = nn.kneighbors(user)
    # rec=pd.DataFrame(df.iloc[ind[0][0]]).T
    for i in range(3):
        links.append(df.iloc[ind[0][i]].link)
        headlines.append(df.iloc[ind[0][i]].headline)
        ids.append(ind[0][i])
    dist,ind = nn.kneighbors(collab_user)
    for i in range(3):
        links.append(df.iloc[ind[0][i]].link)
        headlines.append(df.iloc[ind[0][i]].headline)
        ids.append(ind[0][i])
        
    return (links,headlines,ids)
    
    
    

In [58]:
links,headlines,ids =  recommend(user)


In [60]:
def read_complete(article_id,user):
    user = user[0] + classifier.predict_proba(vectorizer.transform([ df.iloc[article_id].Text_cleaning]))[0] /2
    return user
    # user = user+preds
    
    

In [62]:
user = read_complete(100,user)